# sparkify: An Apache Cassandra Keyspace

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster

def pd_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

try:
    cluster = Cluster(['127.0.0.1']) # This should make a connection to a Cassandra instance your local machine 
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION = 
        {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
session.set_keyspace('sparkify')

#### Query 1
artist, song title and song's length where sessionId = 338, and itemInSession  = 4


In [8]:
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS artist_song_length (
            sessionId int,
            itemInSession int,
            artist text,
            song text,
            length float,
            PRIMARY KEY (sessionId, itemInSession)
        )
    """)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_length (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
session.row_factory = pd_factory

results = session.execute("""
    SELECT artist, song, length
    FROM artist_song_length
    WHERE
        sessionId = 338 AND
        itemInSession = 4
""")

df = results._current_rows
df

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


#### Query 2
name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS artist_song_user (
            userId int,
            sessionId int,
            itemInSession int,
            artist text,
            song text,
            firstName text,
            lastName text,
            PRIMARY KEY ((userId, sessionId), itemInSession)
        )
    """)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO artist_song_user (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

session.row_factory = pd_factory

results2 = session.execute("""
    SELECT artist, song, firstName, lastName
    FROM artist_song_user
    WHERE
        userId=10 AND
        sessionId=182
    ORDER BY itemInSession
""")

df2 = results2._current_rows 
df2

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### Query 3
user name (first and last) for song 'All Hands Against His Own'

In [12]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS user (
            song text,
            firstName text,
            lastName text,
            userId int,
            PRIMARY KEY (song, firstName, lastName, userId)
        )
    """)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO user (song, firstName, lastName, userId)"
        query = query + " VALUES (%s, %s, %s, %s);"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

session.row_factory = pd_factory

results3 = session.execute("""
    SELECT firstName, lastName
    FROM user
    WHERE
        song = 'All Hands Against His Own'
""")

df3 = results3._current_rows 
df3
                    

,firstname,lastname
0,Jacqueline,Lynch
1,Sara,Johnson
2,Tegan,Levine


### Drop the tables before closing out the sessions

In [13]:
try:
    session.execute("DROP TABLE artist_song_length")
    session.execute("DROP TABLE artist_song_user")
    session.execute("DROP TABLE user")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [14]:
session.shutdown()
cluster.shutdown()